# QIIME 2 enables comprehensive end-to-end analysis of diverse microbiome data and comparative studies with publicly available data

this is a QIIME 2 Artifact CLI notebook which replicated analyses in the QIIME 2 protocol

**environment:** qiime2-2020.2

In [ ]:
pip install --upgrade c-lasso

In [ ]:
pip install c-lasso

pip install zarr

pip install plotly

In [2]:
cd ..

In [5]:
ls

Figures			git.code-workspace	q2-types
IRP-framework		q2-classo
c-lasso			q2-sample-classifier


In [ ]:
python setup.py install

pip install -e .

In [3]:
qiime dev refresh-cache

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.


In [6]:
cd q2-classo/example/Data

## Upload data

## Taxonomic classification

## log-contrast processing

In [7]:
qiime classo transform-features \
     --p-transformation clr \
     --p-coef 0.5 \
     --i-features table.qza \
     --o-x xclr

Saved FeatureTable[Design] to: xclr.qza


## Tree aggregation

In [8]:
qiime classo add-taxa \
	--i-features xclr.qza  \
	--i-taxa taxonomy.qza \
	--o-x xtaxa --o-aweights wtaxa

Saved FeatureTable[Design] to: xtaxa.qza
Saved Weights to: wtaxa.qza


## Add covariates

In [9]:
qiime classo add-covariates \
    --i-features xtaxa.qza \
    --m-covariates-file sample-metadata-complete.tsv \
    --p-to-add host_body_mass_index host_age \
    --o-new-features xcovariates \
    --o-new-c ccovariates \
    --verbose

Saved FeatureTable[Design] to: xcovariates.qza
Saved ConstraintMatrix to: ccovariates.qza


## Split table

Split data into training and testing sets : 

In [10]:
qiime sample-classifier split-table \
	--i-table xtaxa.qza \
	--m-metadata-file sample-metadata-complete.tsv \
	--m-metadata-column sCD14  \
	--p-test-size 0.2 \
	--p-random-state 123 \
	--p-stratify False \
	--o-training-table xtraining \
	--o-test-table xtest

Saved FeatureTable[Design] to: xtraining.qza
Saved FeatureTable[Design] to: xtest.qza


## Regression task 

Apply classo to the training set to solve the linear regression problem : 

In [11]:
qiime classo regress  \
    --i-features xtraining.qza \
    --i-weights wtaxa.qza \
    --m-y-file sample-metadata-complete.tsv \
    --m-y-column sCD14  \
    --p-concomitant False \
    --p-stabsel False \
    --p-cv True \
    --p-path True \
    --p-lamfixed True \
    --p-stabsel-threshold 0.5 \
    --p-cv-seed 1 \
    --p-cv-one-se False \
    --o-result regresstaxa

Saved CLASSOProblem to: regresstaxa.qza


## Classification task

In [20]:
qiime classo classify  \
    --i-features xtraining.qza \
    --i-weights wtaxa.qza \
    --m-y-file sample-metadata-complete.tsv \
    --m-y-column HIV_serostatus  \
    --p-stabsel False \
    --p-cv True \
    --p-path True \
    --p-lamfixed True \
    --p-stabsel-threshold 0.5 \
    --p-cv-seed 1 \
    --p-cv-one-se False \
    --o-result classifytaxa \
    --verbose

129 156
/Users/lsimpson/anaconda3/envs/qiime2-2020.11/lib/python3.6/zipfile.py:1378: UserWarning:

Duplicate name: 'data/.zattrs'

/Users/lsimpson/anaconda3/envs/qiime2-2020.11/lib/python3.6/zipfile.py:1378: UserWarning:

Duplicate name: 'formulation/.zattrs'

/Users/lsimpson/anaconda3/envs/qiime2-2020.11/lib/python3.6/zipfile.py:1378: UserWarning:

Duplicate name: 'model_selection/PATHparameters/formulation/.zattrs'

/Users/lsimpson/anaconda3/envs/qiime2-2020.11/lib/python3.6/zipfile.py:1378: UserWarning:

Duplicate name: 'model_selection/PATHparameters/.zattrs'

/Users/lsimpson/anaconda3/envs/qiime2-2020.11/lib/python3.6/zipfile.py:1378: UserWarning:

Duplicate name: 'model_selection/.zattrs'

/Users/lsimpson/anaconda3/envs/qiime2-2020.11/lib/python3.6/zipfile.py:1378: UserWarning:

Duplicate name: 'model_selection/CVparameters/formulation/.zattrs'

/Users/lsimpson/anaconda3/envs/qiime2-2020.11/lib/python3.6/zipfile.py:1378: UserWarning:

Duplicate name: 'model_selection/CVparameters

## Prediction 

In [13]:
qiime classo predict \
    --i-features xtest.qza \
    --i-problem regresstaxa.qza \
    --o-predictions regress-predictions.qza

Saved CLASSOProblem to: regress-predictions.qza


In [14]:
qiime classo predict \
    --i-features xtest.qza \
    --i-problem classifytaxa.qza \
    --o-predictions classify-predictions.qza

Saved CLASSOProblem to: classify-predictions.qza


## Visualization

In [15]:
qiime classo summarize \
  --i-problem regresstaxa.qza \
  --i-taxa taxonomy.qza \
  --i-predictions regress-predictions.qza \
  --o-visualization regresstaxa.qzv

Saved Visualization to: regresstaxa.qzv


In [16]:
qiime classo summarize \
  --i-problem classifytaxa.qza \
  --i-taxa taxonomy.qza \
  --i-predictions classify-predictions.qza \
  --o-visualization classifytaxa.qzv

labels :  ['intercept' 'k__Bacteria' 'k__Archaea' ...
 '57f2c50cfb1e98bfb5ea39cc9f9e559a' '953235e90a7dfa91d15469b8de8ef422'
 '62a761d4144e1ab547dcc3d7589c4983']
C= [[1. 1. 1. ... 1. 1. 1.]]
weights :  [5.74712644e-04 5.00000000e-01 1.44508671e-03 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00]
pred 31 sample 31
Saved Visualization to: classifytaxa.qzv


In [17]:
qiime tools view regresstaxa.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [ ]:
qiime tools view classifytaxa.qzv

Alternatively, one can drag&drop the file problemtaxa.qzv on : https://view.qiime2.org
Thanks to this alternative, one can also track the workflow that the qiime2 artifact did. 